# Compute the NLL metric before and after tracking

In [1]:
import numpy as np
import torch
import math
import os
import properscoring as ps
import warnings
warnings.filterwarnings('ignore')

In [2]:
before_path = "../det/check/check_loss_two_step_center_rh_ind/"
after_path = "./byte_tracker/cov/"
after_cfk_path = after_path + "cfk/"
after_nlla_path = after_path + "nlla/"
after_origin_path = after_path + "origin/"
after_cfk_nlla_path = after_path + "cfk_nlla/"
after_cfk_wcp_path = after_path + "cfk_wcp/"
after_nlla_wcp_path = after_path + "nlla_wcp/"
after_cfk_nlla_wcp_path = after_path + "cfk_nlla_wcp/"
gt_path = "./TrackEval/data/gt/mot_challenge/"
scene_idxes_file_path = "../utils/test_scenes.txt"
var_cp_dict = {"upperbound": [156.3751341925818, 65.24260517866209, 2785.7870238883734, 26.43169171689354], "disco": [214.13191750615232, 77.15047953920649, 2503.1216487206066, 24.387592952924123], "lowerbound": [268.0828005828979, 93.20827260609053, 18277.14814591181, 40.319136673153885], "colowerbound": [479.0016415962182, 408.52224932503424, 26176.90492584989, 39.972214292750124]}

In [3]:
def convert_bbox_to_z(bbox):
    """
    Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
      [x,y,r,h] where x,y is the centre of the box and r is
      the aspect ratio
    """
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = bbox[0] + w / 2.0
    y = bbox[1] + h / 2.0
    if float(h) < 0.00001:
        print([x, y, w, h])
        print(bbox)
    r = w / float(h)
    return np.array([x, y, r, h])

def split_data_by_frame(data):
    res = []
    start = 0
    if data.shape[0] == 0:
        return res
    if len(data.shape) == 1:
        res.append(data[np.newaxis, :])
        return res
    for i in range(1,data.shape[0]):
        if data[i][0] != data[start][0]:
            res.append(data[start:i,:])
            start = i
    res.append(data[start:,:])
    return res

def iou_batch(bb_test, bb_gt):
    """
    From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
    """
    bb_gt = np.expand_dims(bb_gt, 0)
    bb_test = np.expand_dims(bb_test, 1)

    xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    w = np.maximum(0.0, xx2 - xx1)
    h = np.maximum(0.0, yy2 - yy1)
    wh = w * h
    o = wh / (
        (bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])
        + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1])
        - wh
    )
    return o

def linear_assignment(cost_matrix):
    try:
        import lap

        _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
        return np.array([[y[i], i] for i in x if i >= 0])  #
    except ImportError:
        from scipy.optimize import linear_sum_assignment

        x, y = linear_sum_assignment(cost_matrix)
        return np.array(list(zip(x, y)))

def pre_process_bbox(datas):
    """
    make the bbox be [x1,y1,x2,y2] where [x1,y1] is the left bottom and [x2,y2] is the right top
    """
    for i in range(datas.shape[0]):
        small = [min(datas[i][0], datas[i][2]), min(datas[i][1], datas[i][3])]
        large = [max(datas[i][0], datas[i][2]), max(datas[i][1], datas[i][3])]
        datas[i][0] = small[0]
        datas[i][1] = small[1]
        datas[i][2] = large[0]
        datas[i][3] = large[1]
    return datas

def remove_error_bbox(dets):
    res = []
    for i in range(dets.shape[0]):
        if dets[i, 0] == dets[i, 2] or dets[i, 1] == dets[i, 3]:
            continue
        res.append(list(dets[i]))
    return np.array(res)

def convert_bboxs_to_zs(datas):
    for i in range(datas.shape[0]):
        datas[i, :4] = convert_bbox_to_z(datas[i,:4])
    return datas
    
def compute_nll(dets, gts, mode, is_before_tracking, iou_threshold, is_use_cp_cov):
    dets = pre_process_bbox(dets[:,2:])
    gts = pre_process_bbox(gts[:,2:])
    dets = remove_error_bbox(dets)
    iou_matrix = iou_batch(dets, gts)
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0, 2))
    if len(matched_indices) == 0:
        print("Error length of matched_indices if zero!")
    if iou_threshold >= 0.5:
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] >= iou_threshold:
                matches.append([m[0], m[1]])
        matched_indices = np.array(matches)
    if len(np.array(matched_indices).shape) < 2:
        return [], matched_indices
    matched_dets = dets[matched_indices[:,0]]
    matched_gts = gts[matched_indices[:,1]]
    pred = torch.from_numpy(convert_bboxs_to_zs(matched_dets[:,:4]))
    target = torch.from_numpy(convert_bboxs_to_zs(matched_gts[:,:4]))
    if is_before_tracking:
        cov = np.exp(matched_dets[:,8:])
    else:
        cov = matched_dets[:,8:]
    if is_use_cp_cov:
        cp_cov = np.array(var_cp_dict[mode])
        cov = cov * cp_cov
    cov = torch.from_numpy(cov)
    cov = torch.clamp(cov, min=1e-6)
    std = torch.sqrt(cov)
    predicted_normal_dists = torch.distributions.normal.Normal(pred, std)
    negative_log_prob = - predicted_normal_dists.log_prob(target)
    negative_log_prob = torch.clamp(negative_log_prob, min = 0)
    negative_log_prob = torch.sum(negative_log_prob,axis=1)/4
    # if sum(negative_log_prob) > 10*7:
    #     print(dets)
    #     print(gts)
    #     print(matched_indices)
    #     print(negative_log_prob)
    #     print(iou_matrix)
    #     print(matched_dets)
    #     print(matched_gts)
    #     return [], []
    return negative_log_prob.tolist(), matched_indices

def compute_nll_for_mode(mode, det_path, is_before_tracking, iou_threshold, is_use_cp_cov):
    upper_path = det_path + mode + "/no_rsu/"
    scene_idxes_file = open(scene_idxes_file_path, "r")
    scene_idxes = [int(line.strip()) for line in scene_idxes_file]
    #print(scene_idxes)
    nll_res = []
    for agent in range(1,6):
        det_files_path = upper_path + "tracking" + str(agent)
        gt_files_path = gt_path + "V2X-test" + str(agent)
        for scene in scene_idxes:

            if is_before_tracking:
                det_scene_file = os.path.join(det_files_path, "det_" + str(scene) + ".txt")
            else:
                det_scene_file = os.path.join(det_files_path, str(scene) + ".txt")
            gt_scene_file = os.path.join(gt_files_path, str(scene), "gt/gt.txt")
            det_datas = np.loadtxt(det_scene_file, delimiter=",")
            gt_datas = np.loadtxt(gt_scene_file, delimiter=",")
            if len(gt_datas) == 0 or len(det_datas) == 0:
                continue
            det_datas = split_data_by_frame(det_datas)
            gt_datas = split_data_by_frame(gt_datas)
            det_idx = 0
            for gt_idx in range(len(gt_datas)):
                if det_idx >= len(det_datas):
                    break
                if det_datas[det_idx][0][0] != gt_datas[gt_idx][0][0]:
                    #print(f"Error idx not match {scene} {agent} {det_idx}, {gt_idx} {det_datas[det_idx][0][0]} {gt_datas[gt_idx][0][0]}")
                    continue
                if gt_idx >= len(gt_datas):
                    print(f"gt_idx out of range, {gt_idx} {det_idx}")
                    break
                nll, matched_indices = compute_nll(det_datas[det_idx], gt_datas[gt_idx], mode, is_before_tracking, iou_threshold, is_use_cp_cov)
                # if nll == []:
                #     print(det_scene_file)
                #     print(det_datas[det_idx])
                #     return 0
                nll_res.extend(nll)
                det_idx += 1
    nll_mean = sum(nll_res) / len(nll_res)
    return nll_mean

def show_one_result(path, is_before_tracking=True, iou_threshold = 0.5, is_use_cp_cov=True):
    mode_list = ["upperbound", "disco","lowerbound", "colowerbound"]
    for mode in mode_list:
        nll = compute_nll_for_mode(mode, path, is_before_tracking, iou_threshold, is_use_cp_cov)
        print(f"NLL for {mode} is {nll}")

## before tracking

In [4]:
mode = "upperbound"
nll = compute_nll_for_mode(mode, before_path, True, 0.5, True)
print(f"NLL for {mode} is {nll}")

NLL for upperbound is 43.64795034429603


In [5]:
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 43.64795034429603
NLL for disco is 32.500092396658836
NLL for lowerbound is 68.94076520160502
NLL for colowerbound is 956.2331997269524


In [6]:
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 18007.087716853195
NLL for disco is 5407.783418798304
NLL for lowerbound is 4521.834326607164
NLL for colowerbound is 115168.72813619398


In [7]:
show_one_result(before_path, True, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 23.471067282780474
NLL for disco is 19.252723153385052
NLL for lowerbound is 11.78332301955025
NLL for colowerbound is 60.75091831933332


In [8]:
show_one_result(before_path, True, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 8711.60262684237
NLL for disco is 1989.7740744347213
NLL for lowerbound is 1149.3928438755854
NLL for colowerbound is 13517.801753134167


In [9]:
var_cp_dict = {"upperbound": [10,10,10,10], "disco": [10,10,10,10], "lowerbound": [10,10,10,10], "colowerbound": [10,10,10,10]}
show_one_result(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 1800.6133612632095
NLL for disco is 539.7605350308831
NLL for lowerbound is 460.61643586709266
NLL for colowerbound is 12070.567262646957


In [10]:
show_one_result(before_path, True, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 870.3419905811145
NLL for disco is 197.9662562209565
NLL for lowerbound is 120.83173882267289
NLL for colowerbound is 1897.8557301179264


## after tracking origin

In [5]:
show_one_result(after_origin_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 125.20177359379358
NLL for disco is 74.75636795657931
NLL for lowerbound is 11.233907746595916


In [6]:
show_one_result(after_origin_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 60.21976926476921
NLL for disco is 10.384476516768254
NLL for lowerbound is 3.357057056450926


## after tracking cfk

In [32]:
show_one_result(after_cfk_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 26.330773410274663
NLL for disco is 24.4282927477489
NLL for lowerbound is 44.13806391046556


In [21]:
show_one_result(after_cfk_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 14.487700458698557
NLL for disco is 16.41231819549025
NLL for lowerbound is 10.262965532542061


In [34]:
show_one_result(after_cfk_wcp_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 12627.398769808899
NLL for disco is 5554.220153995415
NLL for lowerbound is 2413.198522431289


In [35]:
show_one_result(after_cfk_wcp_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 1624.1060303987313
NLL for disco is 1888.4322677093492
NLL for lowerbound is 748.5118266402267


## after tracking nlla

In [9]:
show_one_result(after_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 6062.201114493875
NLL for disco is 81.28181122822285
NLL for lowerbound is 12.3513828926785


In [10]:
show_one_result(after_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 6137.521950537949
NLL for disco is 11.007561204589447
NLL for lowerbound is 3.8936961052324532


In [23]:
show_one_result(after_nlla_wcp_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 125.1106189092142
NLL for disco is 74.62391822484624
NLL for lowerbound is 11.330465651175567


In [24]:
show_one_result(after_nlla_wcp_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 60.20765030497474
NLL for disco is 10.386071545374097
NLL for lowerbound is 3.38768813451858


## after tracking cfk + nlla

In [11]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 29.490438899196835
NLL for disco is 25.301711427987456
NLL for lowerbound is 57.23140681948101
NLL for colowerbound is 1053.6949571139912


In [12]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 17.672873672676403
NLL for disco is 17.569613491676424
NLL for lowerbound is 11.265893531969354
NLL for colowerbound is 106.60307789009637


In [36]:
show_one_result(after_cfk_nlla_wcp_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 12614.369160824344
NLL for disco is 5547.999893241213
NLL for lowerbound is 2447.807312076365


In [37]:
show_one_result(after_cfk_nlla_wcp_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 1623.2255846201076
NLL for disco is 1890.2503609553771
NLL for lowerbound is 785.9194419388125


In [13]:
#cp1
var_cp_dict = {"upperbound": [2.0168343618519997, 20.655256274811332, 0.0065616422113331276, 9.810218760278726], "disco": [1.5844876978208504, 16.85182149486315, 0.007767244933272351, 10.737039123121171], "lowerbound": [0.9496513748504812, 14.405661414225111, 0.0015879443245170666, 6.56055795747982], "colowerbound": [0.9433605467339239, 4.170984537805175, 0.0011905071783342841, 8.553786312580932]}
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 790.8173002237859
NLL for disco is 225.09685639266633
NLL for lowerbound is 21.223065815826025
NLL for colowerbound is 2726.6052428577896


In [14]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 63.51246409204609
NLL for disco is 41.79856141397415
NLL for lowerbound is 3.790857297459536
NLL for colowerbound is 500.80423974913606


In [15]:
#cp2
var_cp_dict = {"upperbound": [5.6827505661910545, 39.22419803102757, 0.2799106202333024, 23.82625371377505], "disco": [5.134614410746115, 38.66801996668446, 0.3031539483223637, 22.899039617727098], "lowerbound": [4.463060508876752, 30.744574219188294, 0.028465764379049255, 18.922759296842013], "colowerbound": [6.611347658813065, 12.712682613242992, 0.3338620534276504, 35.427210831086256]}
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 20.008545312013826
NLL for disco is 6.93993810634455
NLL for lowerbound is 4.046041364416762
NLL for colowerbound is 54.35242205793384


In [16]:
show_one_result(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 2.4103872888472315
NLL for disco is 1.9882966497338894
NLL for lowerbound is 0.9884447863092921
NLL for colowerbound is 5.990936264756574


# CRPS

In [24]:
var_cp_dict = {"upperbound": [156.3751341925818, 65.24260517866209, 2785.7870238883734, 26.43169171689354], "disco": [214.13191750615232, 77.15047953920649, 2503.1216487206066, 24.387592952924123], "lowerbound": [268.0828005828979, 93.20827260609053, 18277.14814591181, 40.319136673153885], "colowerbound": [479.0016415962182, 408.52224932503424, 26176.90492584989, 39.972214292750124]}
def compute_crps(dets, gts, mode, is_before_tracking, iou_threshold, is_use_cp_cov):
    dets = pre_process_bbox(dets[:,2:])
    gts = pre_process_bbox(gts[:,2:])
    dets = remove_error_bbox(dets)
    iou_matrix = iou_batch(dets, gts)
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0, 2))
    if len(matched_indices) == 0:
        print("Error length of matched_indices if zero!")
    if iou_threshold >= 0.5:
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] >= iou_threshold:
                matches.append([m[0], m[1]])
        matched_indices = np.array(matches)
    if len(np.array(matched_indices).shape) < 2:
        return [], matched_indices
    matched_dets = dets[matched_indices[:,0]]
    matched_gts = gts[matched_indices[:,1]]
    pred = convert_bboxs_to_zs(matched_dets[:,:4])
    target = convert_bboxs_to_zs(matched_gts[:,:4])
    if is_before_tracking:
        cov = np.exp(matched_dets[:,8:])
    else:
        cov = matched_dets[:,8:]
    if is_use_cp_cov:
        cp_cov = np.array(var_cp_dict[mode])
        cov = cov * cp_cov
    cov = np.clip(cov, 1e-6, None)
    std = np.sqrt(cov)
    crps = ps.crps_gaussian(target, 
                               pred,
                               std)
    return np.mean(crps, axis=1).tolist(), matched_indices

def compute_crps_for_mode(mode, det_path, is_before_tracking, iou_threshold, is_use_cp_cov):
    upper_path = det_path + mode + "/no_rsu/"
    scene_idxes_file = open(scene_idxes_file_path, "r")
    scene_idxes = [int(line.strip()) for line in scene_idxes_file]
    #print(scene_idxes)
    nll_res = []
    for agent in range(1,6):
        det_files_path = upper_path + "tracking" + str(agent)
        gt_files_path = gt_path + "V2X-test" + str(agent)
        for scene in scene_idxes:
            if is_before_tracking:
                det_scene_file = os.path.join(det_files_path, "det_" + str(scene) + ".txt")
            else:
                det_scene_file = os.path.join(det_files_path, str(scene) + ".txt")
            gt_scene_file = os.path.join(gt_files_path, str(scene), "gt/gt.txt")
            det_datas = np.loadtxt(det_scene_file, delimiter=",")
            gt_datas = np.loadtxt(gt_scene_file, delimiter=",")
            if len(gt_datas) == 0 or len(det_datas) == 0:
                continue
            det_datas = split_data_by_frame(det_datas)
            gt_datas = split_data_by_frame(gt_datas)
            det_idx = 0
            for gt_idx in range(len(gt_datas)):
                if det_idx >= len(det_datas):
                    break
                if det_datas[det_idx][0][0] != gt_datas[gt_idx][0][0]:
                    #print(f"Error idx not match {scene} {agent} {det_idx}, {gt_idx} {det_datas[det_idx][0][0]} {gt_datas[gt_idx][0][0]}")
                    continue
                if gt_idx >= len(gt_datas):
                    print(f"gt_idx out of range, {gt_idx} {det_idx}")
                    break
                nll, matched_indices = compute_crps(det_datas[det_idx], gt_datas[gt_idx], mode, is_before_tracking, iou_threshold, is_use_cp_cov)
                # if nll == []:
                #     print(det_scene_file)
                #     print(det_datas[det_idx])
                #     return 0
                nll_res.extend(nll)
                det_idx += 1
    nll_mean = sum(nll_res) / len(nll_res)
    return nll_mean

def show_one_result_crps(path, is_before_tracking=True, iou_threshold = 0.5, is_use_cp_cov=True):
    mode_list = ["upperbound", "disco", "lowerbound", "colowerbound"]
    for mode in mode_list:
        nll = compute_crps_for_mode(mode, path, is_before_tracking, iou_threshold, is_use_cp_cov)
        print(f"NLL for {mode} is {nll}")

In [21]:
show_one_result_crps(before_path, True, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 0.4080248716609266
NLL for disco is 0.40717371825925913
NLL for lowerbound is 0.6139440500976983
NLL for colowerbound is 0.7248706492372927


In [22]:
show_one_result_crps(before_path, True, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 0.35393592026537857
NLL for disco is 0.37349087251094154
NLL for lowerbound is 0.37265631465724475
NLL for colowerbound is 0.3963018535195489


In [25]:
show_one_result_crps(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 0.38112657321147286
NLL for disco is 0.3758511544557144
NLL for lowerbound is 0.5957584860668782
NLL for colowerbound is 0.7067062867677941


In [26]:
show_one_result_crps(before_path, True, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 0.32785109001124263
NLL for disco is 0.34286798810929925
NLL for lowerbound is 0.35776305326689367
NLL for colowerbound is 0.3818840770386424


In [43]:
var_cp_dict = {"upperbound": [1,1,1,1], "disco": [1,1,1,1], "lowerbound": [1,1,1,1], "colowerbound": [1, 10, 0.01, 10]}
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 0.3018026409488238
NLL for disco is 0.3180079052483192
NLL for lowerbound is 0.41209860381471847
NLL for colowerbound is 0.7131665629006314


In [44]:
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 0.2757033159557821
NLL for disco is 0.30786995806703615
NLL for lowerbound is 0.310081266360668
NLL for colowerbound is 0.3621501393770264


In [43]:
show_one_result_crps(after_cfk_nlla_wcp_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.33328646433974984
NLL for upperbound is 0.312034388183891
NLL for lowerbound is 0.4068239378390565


In [44]:
show_one_result_crps(after_cfk_nlla_wcp_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.32393537311172754
NLL for upperbound is 0.2860397491937557
NLL for lowerbound is 0.31136575282299306


In [29]:
show_one_result_crps(after_origin_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for upperbound is 0.26602671855205295
NLL for disco is 0.29699630329003535
NLL for lowerbound is 0.28731373436471996
NLL for colowerbound is 0.39717483747756993


In [30]:
show_one_result_crps(after_origin_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for upperbound is 0.29558470373650764
NLL for disco is 0.30720742583481137
NLL for lowerbound is 0.3852514384924846
NLL for colowerbound is 0.7588563619818461


In [12]:
show_one_result_crps(after_cfk_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.31041026511366365
NLL for upperbound is 0.29076805311055326
NLL for lowerbound is 0.3940620279075051


In [13]:
show_one_result_crps(after_cfk_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.30071852072848637
NLL for upperbound is 0.26539920132392575
NLL for lowerbound is 0.30010412018881394


In [45]:
show_one_result_crps(after_cfk_wcp_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.33211376392623626
NLL for upperbound is 0.3116887897428072
NLL for lowerbound is 0.40427279568120406


In [46]:
show_one_result_crps(after_cfk_wcp_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.3227192998320848
NLL for upperbound is 0.28578126617371225
NLL for lowerbound is 0.30900291226228227


In [14]:
show_one_result_crps(after_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.3165169661603565
NLL for upperbound is 0.30523531448623475
NLL for lowerbound is 0.3988005262564758


In [15]:
show_one_result_crps(after_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.30344164895458337
NLL for upperbound is 0.2756183241126243
NLL for lowerbound is 0.2947408333019423


In [47]:
show_one_result_crps(after_nlla_wcp_path, False, iou_threshold=0.5, is_use_cp_cov=False)

NLL for disco is 0.3080300533567466
NLL for upperbound is 0.29573923388258844
NLL for lowerbound is 0.38637524730972794


In [48]:
show_one_result_crps(after_nlla_wcp_path, False, iou_threshold=0.7, is_use_cp_cov=False)

NLL for disco is 0.2978604181251968
NLL for upperbound is 0.2661644190736253
NLL for lowerbound is 0.28804841219611216


In [31]:
#cp1
var_cp_dict = {"upperbound": [2.0168343618519997, 20.655256274811332, 0.0065616422113331276, 9.810218760278726], "disco": [1.5844876978208504, 16.85182149486315, 0.007767244933272351, 10.737039123121171], "lowerbound": [0.9496513748504812, 14.405661414225111, 0.0015879443245170666, 6.56055795747982], "colowerbound":[0.9433605467339239, 4.170984537805175, 0.0011905071783342841, 8.553786312580932]}
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 0.27128180217931547
NLL for disco is 0.2820462212845164
NLL for lowerbound is 0.37057126417101566
NLL for colowerbound is 0.7127095524250155


In [32]:
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 0.24525212992355208
NLL for disco is 0.2717350683020294
NLL for lowerbound is 0.2690068640609669
NLL for colowerbound is 0.3603005377840122


In [33]:
#cp2
var_cp_dict = {"upperbound": [5.6827505661910545, 39.22419803102757, 0.2799106202333024, 23.82625371377505], "disco": [5.134614410746115, 38.66801996668446, 0.3031539483223637, 22.899039617727098], "lowerbound": [4.463060508876752, 30.744574219188294, 0.028465764379049255, 18.922759296842013], "colowerbound": [6.611347658813065, 12.712682613242992, 0.3338620534276504, 35.427210831086256]}
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 0.2799833163483051
NLL for disco is 0.2857141234003051
NLL for lowerbound is 0.37552595716813714
NLL for colowerbound is 0.736156171915419


In [34]:
show_one_result_crps(after_cfk_nlla_path, False, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 0.25404506221256606
NLL for disco is 0.2754788535427239
NLL for lowerbound is 0.2755813538190116
NLL for colowerbound is 0.3899526624570473


In [19]:
var_cp_dict = {"upperbound": [10,10,10,10], "disco": [10,10,10,10], "lowerbound": [10,10,10,10], "colowerbound": [10,10,10,10]}
show_one_result_crps(before_path, True, iou_threshold=0.5, is_use_cp_cov=True)

NLL for upperbound is 0.39173273107394513
NLL for disco is 0.3909821063044029
NLL for lowerbound is 0.597441010158856
NLL for colowerbound is 0.7114729947454802


In [20]:
show_one_result_crps(before_path, True, iou_threshold=0.7, is_use_cp_cov=True)

NLL for upperbound is 0.3379260413233048
NLL for disco is 0.357450976072768
NLL for lowerbound is 0.3566045180617696
NLL for colowerbound is 0.3832530401134152
